# Исследование надежности заемщиков


## Загрузка данных

Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`. 

In [35]:
import pandas as pd

Не показан импорт рабочего датасета в соответствии с политикой конфиденциальности ЯП.

Первые 20 строчек датафрейма `data`:

In [4]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем основную информацию о датафрейме с помощью метода `info()`:

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Выведем количество пропущенных значений для каждого столбца:

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце мы обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом:

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`:

In [7]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях:

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть:

Выведем перечень уникальных значений столбца `children`:

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`:

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены:

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`:

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов:

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`:

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверим остальные столбцы:

In [15]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их:

In [16]:
data.duplicated().sum()

71

In [17]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Создадим собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`:

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут правильно определить категорию.**

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследуем данные и ответим на вопросы

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [23]:
children_grouped = data.groupby('children').agg({
    'children': 'count',
    'debt': 'sum'
})

display(children_grouped)

,children,debt
children,,
0,14091,1063
1,4808,444
2,2052,194
3,330,27
4,41,4
5,9,0


In [24]:
children_grouped['debt_persent'] = children_grouped['debt'] / children_grouped[
    'children'] * 100

display(children_grouped)

,children,debt,debt_persent
children,,,
0,14091,1063,7.543822
1,4808,444,9.234609
2,2052,194,9.454191
3,330,27,8.181818
4,41,4,9.756098
5,9,0,0.000000


In [25]:
children_grouped.columns = ['children_amount', 'debt_sum',
                            'debt_persent']  #Переименовала столбцы

children_grouped.sort_values(
    by='debt_persent', ascending=False
)  #сортировка по убыванию процента невозврата кредита в срок

,children_amount,debt_sum,debt_persent
children,,,
4,41,4,9.756098
2,2052,194,9.454191
1,4808,444,9.234609
3,330,27,8.181818
0,14091,1063,7.543822
5,9,0,0.000000


**Вывод:** 

 - Процент **невозврата** кредита **повышается** с увеличением **количества детей**
 - По категории с 5-ю детьми недостаточно данных для анализа

####  Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
family_status_connection = data.groupby('family_status').agg({
    'debt':
    'sum',
    'family_status':
    'count'
})
display(family_status_connection)

,debt,family_status
family_status,,
Не женат / не замужем,273,2796
в разводе,84,1189
вдовец / вдова,63,951
гражданский брак,385,4134
женат / замужем,927,12261


In [28]:
family_status_connection['debt_persent'] = family_status_connection[
    'debt'] / family_status_connection['family_status'] * 100
display(family_status_connection)

,debt,family_status,debt_persent
family_status,,,
Не женат / не замужем,273,2796,9.763948
в разводе,84,1189,7.064760
вдовец / вдова,63,951,6.624606
гражданский брак,385,4134,9.313014
женат / замужем,927,12261,7.560558


In [29]:
display(
    family_status_connection.sort_values(by='debt_persent', ascending=False))
#сортировка по убыванию процента невозврата кредита в срок

,debt,family_status,debt_persent
family_status,,,
Не женат / не замужем,273,2796,9.763948
гражданский брак,385,4134,9.313014
женат / замужем,927,12261,7.560558
в разводе,84,1189,7.064760
вдовец / вдова,63,951,6.624606


**Вывод:** 

- У официально **не семейных** людей процент **невозврата** выше

####  Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [30]:
total_income_grouped = data.groupby('total_income_category').agg({
    'debt':
    'sum',
    'total_income':
    'count'
})
display(total_income_grouped)

,debt,total_income
total_income_category,,
A,2,25
B,354,5014
C,1353,15921
D,21,349
E,2,22


In [33]:
total_income_grouped['debt_persent'] = total_income_grouped[
    'debt'] / total_income_grouped['total_income'] * 100
display(total_income_grouped.sort_values(by='debt_persent', ascending=False))

,debt,total_income,debt_persent
total_income_category,,,
E,2,22,9.090909
C,1353,15921,8.498210
A,2,25,8.000000
B,354,5014,7.060231
D,21,349,6.017192


**Вывод :** 

- Процент невозврата приблизительно одинаков у категорий Е и С, хотя категория E очень малочислена. Возможно банку следует обратить внимание и на эту группу заемщиков и повысить их количество(что возможно увеличит популярность банка среди населения).

- Также малочисленной группой заемщиков является группа D, но у них самый низкий процент задолженности.




#### Как разные цели кредита влияют на его возврат в срок?

In [34]:
purpose_grouped = data.groupby('purpose_category').agg({
    'debt':
    'sum',
    'purpose_category':
    'count'
})
purpose_grouped['debt_persent'] = purpose_grouped['debt'] / purpose_grouped[
    'purpose_category'] * 100
display(purpose_grouped.sort_values(by='debt_persent', ascending=False))

,debt,purpose_category,debt_persent
purpose_category,,,
операции с автомобилем,400,4279,9.347978
получение образования,369,3988,9.252758
проведение свадьбы,183,2313,7.911803
операции с недвижимостью,780,10751,7.255139


**Вывод:** 

- Самый большой процент невозврата кредита при операциях с автомобилем.
- Самый маленький процент невозврата кредита при операциях с недвижимостью.
- Рисковой категорией по возврату кредита наряду с автомобилями является категория получения образования.


####  Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* 

Пропуски в исходных данных появляются по нескольким причинам:
+ Пользователь намеренно не указывает каких-либо данных о себе.
+ Система дает cбой при чтении файла с данными.
+ Пропуски оставляют намеренно, чтобы после заполнить их какими либо спецсимволами автоматически.


#### Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* 
```
Заполнять пропуски медианным значением в количественных переменных - лучший вариант, т.к на значение медианы 
не влияют выдающиеся числовые значения, если они присутствуют в выборке.
```

## Общий вывод

**В процессе исследования и обработки данных от кредитного отдела банка было установлено, что:**


+ **Процент невозврата кредита повышается с увеличением количества детей.**
+ **У официально не семейных людей выше процент невозврата кредита в срок.**

+ **Процент возвращения кредита в срок следует рассматривать с учетом количества заемщиков в каждой группе. Например для адекватной оценки процента возврата кредита по группам Е (доход <30 тыс.руб.) и А(доход > 1млн.руб.) в них слишком мало людей. Банку следует обратить внимание на группу E,т.к процент невозврата кредита у нее близок к проценту невозврата кредита у самой многочисленной группы С(доход 50-200 тыс.руб.). Возможно, если увеличить количество заемщиков в этой группе, которым банк будет выдавать кредит, увеличиться доход банка и популярность среди населения.**

+ **Самый большой процент невозврата кредита при операциях с автомобилем и он сравним с показателем в группе кредитов на образование, а самый маленький - при операциях с недвижимостью.**
